# LLM-human collaborative annotation 
This notebook illustrates the integration of Large Language Models (LLMs) into the Labeler framework. In this framework, LLMs serve as annotators, and human verification is used to validate the annotation results. Initially, we demonstrate this integration with OpenAI's GPT models and completion APIs.

Users can register agents by specifying model configurations and prompt configurations, select a subset, and run the job. Labeler takes care of the following tasks:

* Interfacing with OpenAI and handling errors.
* Executing LLM models and persisting the results.
* Providing flexible search capabilities to support human verification and downstream applications.


# 1. Setup
## 1.1 Authentication and Labeler project connection

In [ ]:
from labeler_client import Authentication
auth = Authentication(project="llm_demo")

In [ ]:
from labeler_client import Service
service = Service(project='llm_demo', auth=auth)

## 1.2 Data import and schema config 
(For new project only, skip if already set)

In [ ]:
### (skip) import data and create schema
# import pandas as pd
# df = pd.read_csv('tweets.csv')
# service.import_data_df(df, column_mapping={
#     'id':'id',
#     'content':'content'
# })

# service.get_schemas().set_schemas({
#     'label_schema': [
#         {
#             "name": "sentiment",
#             "level": "record",
#             "options": [
#                 { "value": "pos", "text": "positive" },
#                 { "value": "neu", "text": "neutral" },
#                 { "value": "neg", "text": "negative" },
#             ]
#         }
#     ]
# })

## 1.3 Review labeling schema

In [ ]:
### review schema
schema = service.get_schemas().value(active=True)
print(schema)

# 2. LLM Annotation
## 2.1 Config model and prompt template

In [ ]:
model_config = {'model': 'text-davinci-003', 'temperature': 0, 'n': 1} # define model configs
# OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] # provide your open ai api key here
# OPENAI_ORGANIZATION = os.environ['OPENAI_ORGANIZATION'] if 'OPENAI_ORGANIZATION' in os.environ else '' # provide an openai organization key if needed

In [ ]:
label_name = 'sentiment'

In [ ]:
from labeler_client.prompt import PromptTemplate
prompt_template = PromptTemplate(label_schema=schema[0]['schemas']['label_schema'], label_names=[label_name])
prompt_template.preview(records=['[sample input]', 'Megagon Labs is located in Mountain View.'])

In [ ]:
prompt_template.get_template()

## 2.2 Register an agent with service

In [ ]:
import os
from labeler_client.controller import Controller
controller = Controller(service, auth)

In [ ]:
agent_uuid = controller.create_agent(model_config, prompt_template)

## 2.3 Run an LLM annotation job on subsets
**!Make sure OPENAI_API_KEY is set as an env var.**

In [ ]:
# test to make sure OPENAI_API_KEY is set, be carefule not to commit the key
import os
#print(os.environ['OPENAI_API_KEY'])

In [ ]:
subset = service.search(keyword="delay", limit=10, start=0)
# subset.show({'view': 'table'}) # for data visualization
job_uuid = controller.run_job(agent_uuid, subset, label_name)

In [ ]:
# Test: second job with same agent, different subset
subset2 = service.search(keyword='great', limit=10, start=0)
job_uuid2 = controller.run_job(agent_uuid, subset2, label_name)

## 2.4 List agents & jobs

In [ ]:
agents = controller.list_my_agents()
agents

In [ ]:
d = controller.get_agent_by_uuid(agent_uuid)
d

In [ ]:
# controller.list_jobs_of_agent(agent_uuid)
job_list = controller.list_jobs('agent_uuid', [agent_uuid])
# controller.list_jobs('issued_by', [])
job_list

# 3. Human Verification
## 3.1 prepare for verification

In [ ]:
# As an example, set to first job in the list from the previous query
# replace with job_id to verify
job_id = job_list[0][0] 
# Optional verified filter to retrieve a subset with or without verifications
verf_subset = service.search_by_job(job_id=job_id, verified=False)


## 3.2 Verify LLM output
In the 'sentiment' column, the labels are generated by the LLM. If any of these labels need correction, simply update them in the user interface. For accurate labels, leave them unmodified. Once you have verified the subset, select all and click the 'Verify' button in the upper-left corner.

In [ ]:
verf_subset.show({"title": "Verification", "view": "table", "mode": "verifying"})

## 3.3 Retrieve Verification Annotations
The current version supports only programmatic retrieval of previous verifications, 

In [ ]:
verf_subset.get_verification_annotations(
    label_name="sentiment", 
    label_level="record",
    annotator=job_id
)

In [ ]:
# further filter by type of verification(CONFIRMS|CORRECTS)
# CONFIMS:  where the verification confirms the original label
# CORRECTS: where the verification is different from the original label
verf_subset.get_verification_annotations(
    label_name="sentiment", 
    label_level="record",
    annotator=job_id,
    verified_status='CORRECTS'
)